# Running Pyomo on Google Colab

Keywords: installation

This note notebook shows how to install the basic pyomo package on Google Colab, and then demonstrates the subsequent installation and use of various solvers including

* GLPK
* COIN-OR CBC
* COIN-OR Ipopt
* COIN-OR Bonmin
* COIN-OR Couenne
* COIN-OR Gecode

## Basic installation of Pyomo

We'll do a quiet installation of pyomo using `pip`.  This needs to be done once at the start of each Colab session.

In [2]:
!pip install -q pyomo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.2 MB/s eta 0:00:00


The installation of pyomo can be verified by entering a simple model. We'll use the model again in subsequent cells to demonstrate the installation and execution of various solvers.

In [3]:
#fantasy
import pyomo.environ as pyo
import numpy as np


model = pyo.ConcreteModel()
model.i = np.arange(4) #number of positions
model.j = np.arange(10) #number of  players
model.point = np.random.randint(100,size = (len(model.i),len(model.j)))
model.price = np.random.randint(15,size = (len(model.i),len(model.j)))
model.max_positions = np.array([1,5,5,3])
model.min_positions = np.array([1,3,3,1])
model.x = pyo.Var(model.i,model.j,domain = pyo.Binary)

def Obj(model): #objective function
    return sum(model.point[i,j] * model.x[i,j]  for i in range(len(model.i))    for j in range(len(model.j)))

def Co1 (model): #sum of xi s have to be exactly 11
    return sum(model.x[i,j]   for i in range(len(model.i))  for j in range(len(model.j))) == 11

def Co2(model): #budget constraint
    return sum(model.price[i,j]*model.x[i,j]  for i in range(len(model.i))  for j in range(len(model.j)))<=82

def Co3(model,i): #maximum number of picked players
    return sum(model.x[i,j] for j in range(len(model.j))) <= model.max_positions[i]
def Co4(model,i): #minimum number of picked players
    return sum(model.x[i,j] for j in range(len(model.j))) >= model.min_positions[i]


model.obj = pyo.Objective(rule = Obj,sense=pyo.maximize)
model.Co1 = pyo.Constraint(rule = Co1)
model.Co2 = pyo.Constraint(rule = Co2)
model.Co3 = pyo.Constraint(range(len(model.i)),rule = Co3)
model.Co4 = pyo.Constraint(range(len(model.i)),rule = Co4)
# solver = pyo.SolverFactory('glpk',executable = 'C:\glpk-4.65\w64\glpsol')
# results = solver.solve(model)

# model.display()
model.point,model.price
# for i in range(len(model.i)):
#     for j in range(len(model.j)):
#         print((i,j),model.x[i,j].value)

(array([[65, 10, 61, 38, 80,  9,  1, 99, 89, 65],
        [25, 74, 40,  1, 13, 60, 11, 39, 99,  4],
        [68, 91, 91, 69, 86, 52, 59, 39, 75, 63],
        [39, 47, 85, 70, 63, 35, 71, 39, 59, 64]]),
 array([[ 9,  2, 12,  7,  3, 13, 10,  8,  4, 14],
        [ 4,  2, 11,  9,  4,  9,  0,  5,  8,  8],
        [ 1,  3,  6, 14, 13,  0,  6,  2,  1, 10],
        [10,  9,  1,  1,  9,  5, 12, 14,  9,  8]]))

## GLPK installation

Keywords: GLPK

[GLPK](https://en.wikibooks.org/wiki/GLPK) is a the open-source **G**NU **L**inear **P**rogramming **K**it available for use under the GNU General Public License 3. GLPK is a single-threaded simplex solver generally suited to small to medium scale linear-integer programming problems. It is written in C with minimal dependencies and is therefore highly portable among computers and operating systems. GLPK is often 'good enough' for many examples. For larger problems users should consider higher-performance solvers, such as COIN-OR CBC, that can take advantage of multi-threaded processors.

In [4]:
!apt-get install -y -qq glpk-utils

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 120874 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_5.0-1_amd64.deb ...
Unpacking glpk-utils (5.0-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4b

In [7]:
pyo.SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model).write()

# display solution
model.display
# print('\nProfit = ', model.profit())

# print('\nDecision Variables')
# print('x = ', model.x())
# print('y = ', model.y())

# print('\nConstraints')
# print('Demand  = ', model.demand())
# print('Labor A = ', model.laborA())
# print('Labor B = ', model.laborB())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 901.0
  Upper bound: 901.0
  Number of objectives: 1
  Number of constraints: 10
  Number of variables: 40
  Number of nonzeros: 158
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.005349159240722656
# ----------------------------------------------------------
#   Solution Information
# --------------------------------

<bound method Block.display of <pyomo.core.base.PyomoModel.ConcreteModel object at 0x78ec4198e110>>